In [4]:
from pathlib import Path
import pandas as pd
import seaborn as sns

In [5]:
train_dir = Path(Path.cwd().parent, "data", "raw", "train.csv")
df = pd.read_csv(train_dir)

In [6]:
# What does the data look like?
df.head()

,date,store,item,sales
0,2013-01-01,1,1,13
1,2013-01-02,1,1,11
2,2013-01-03,1,1,14
3,2013-01-04,1,1,13
4,2013-01-05,1,1,10


In [7]:
# How many rows, columns are there?
df.shape

(913000, 4)

In [8]:
# What timerange do we have
df["date"].min(), df["date"].max()

('2013-01-01', '2017-12-31')

In [9]:
# How many unique items per store are there
df.groupby(["store"])["item"].nunique()

store
1     50
2     50
3     50
4     50
5     50
6     50
7     50
8     50
9     50
10    50
Name: item, dtype: int64

In [14]:
df.head()

,date,store,item,sales
0,2013-01-01,1,1,13
1,2013-01-02,1,1,11
2,2013-01-03,1,1,14
3,2013-01-04,1,1,13
4,2013-01-05,1,1,10


In [34]:
total_sales_by_date = df.groupby(['date'], as_index = False).sum(numeric_only = True)[['date', 'sales']]
total_sales_by_date.head()